In [180]:

import pygad

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import joblib

In [ ]:
data = pd.read_csv('dataset_1.csv')

figure_dict = {
    'circle': 0,
    'rectangle': 1
}
data['figure1'] = data['figure1'].map(figure_dict)
data['figure2'] = data['figure2'].map(figure_dict)


X = data.drop('collision', axis=1)
y = data['collision']

# Предобработка
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение диапазонов гиперпараметров
decision_tree_param_ranges = {
    'max_depth': (1, 20),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20)
}

naive_bayes_param_ranges = {
    'var_smoothing': (1e-11, 1e-7)
}

In [182]:
# Целевая функция для Decision Tree
def decision_tree_fitness_function(ga_instance, solution, solution_idx):
    max_depth = int(solution[0])
    min_samples_split = int(solution[1])
    min_samples_leaf = int(solution[2])
    
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

# Целевая функция для Naive Bayes
def naive_bayes_fitness_function(ga_instance, solution, solution_idx):
    var_smoothing = solution[0]
    
    model = GaussianNB(var_smoothing=var_smoothing)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

In [183]:
# Настройка генетического алгоритма для Decision Tree
ga_decision_tree = pygad.GA(
    num_generations=20,
    num_parents_mating=5,
    fitness_func=decision_tree_fitness_function,
    sol_per_pop=10,
    num_genes=3,
    gene_space=[
        {'low': decision_tree_param_ranges['max_depth'][0], 'high': decision_tree_param_ranges['max_depth'][1]},
        {'low': decision_tree_param_ranges['min_samples_split'][0], 'high': decision_tree_param_ranges['min_samples_split'][1]},
        {'low': decision_tree_param_ranges['min_samples_leaf'][0], 'high': decision_tree_param_ranges['min_samples_leaf'][1]}
    ],
    mutation_percent_genes=10
)

# Настройка генетического алгоритма для Naive Bayes
ga_naive_bayes = pygad.GA(
    num_generations=20,
    num_parents_mating=5,
    fitness_func=naive_bayes_fitness_function,
    sol_per_pop=10,
    num_genes=1,
    gene_space=[
        {'low': naive_bayes_param_ranges['var_smoothing'][0], 'high': naive_bayes_param_ranges['var_smoothing'][1]}
    ],
    mutation_percent_genes=10
)

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


In [184]:

# Запуск генетического алгоритма для Decision Tree
print("Дерево решений")
ga_decision_tree.run()
solution_dt, solution_fitness_dt, _ = ga_decision_tree.best_solution()
print(f"Лучшие параметры: max_depth={int(solution_dt[0])}, min_samples_split={int(solution_dt[1])}, min_samples_leaf={int(solution_dt[2])}")
print(f"Лучший Accuracy: {solution_fitness_dt}")

best_dt_model = DecisionTreeClassifier(
    max_depth=int(solution_dt[0]),
    min_samples_split=int(solution_dt[1]),
    min_samples_leaf=int(solution_dt[2]),
    random_state=42
)
joblib.dump(best_dt_model, '1_best_decision_tree_model.pkl')

# Запуск генетического алгоритма для Naive Bayes
print("\nНаивный Байес")
ga_naive_bayes.run()
solution_nb, solution_fitness_nb, _ = ga_naive_bayes.best_solution()
print(f"Лучший параметр: var_smoothing={solution_nb[0]}")
print(f"Лучший Accuracy: {solution_fitness_nb}")

best_nb_model = GaussianNB(
    var_smoothing=float(solution_nb[0])
)
joblib.dump(best_nb_model, '1_best_naive_bayes_model.pkl')

Дерево решений
Лучшие параметры: max_depth=5, min_samples_split=18, min_samples_leaf=1
Лучший Accuracy: 0.8

Наивный Байес
Лучший параметр: var_smoothing=1e-11
Лучший Accuracy: 0.8


['1_best_naive_bayes_model.pkl']